# Playbook: Phishing Incident Triage

**Timestamp:** 2025-08-04 11:00 AM EDT
**Analyst:** Darren Shady
**Incident ID:** (Will be generated by the script)

**Objective:** To perform initial analysis on a reported phishing email to determine its maliciousness, identify key indicators, and recommend initial containment steps.

**CISA Alignment:** Detection & Analysis, Containment.
**MITRE ATT&CK Alignment:**
| Tactic | Technique ID | Technique Name |
|---|---|---|
| Initial Access | T1566 | Phishing |
| Initial Access | T1566.001 | Spearphishing Attachment |
| Initial Access | T1566.002 | Spearphishing Link |

In [ ]:
import json
import re
from datetime import datetime, timezone
import pandas as pd

# Configure pandas to display full URL text
pd.set_option('display.max_colwidth', None)

# --- Configuration (User to customize) ---
KNOWN_BAD_DOMAINS = {"evil-domain.com", "phish-corp.net"}
KNOWN_BAD_IPS = {"198.51.100.55"}
SUSPICIOUS_KEYWORDS = {"login", "verify", "account", "update", "urgent", "password"}

# --- Mock Threat Intelligence Function ---
def query_threat_intel_mock(indicator_type: str, value: str) -> dict:
    """Mocks a threat intelligence query. Replace with actual API calls."""
    print(f"[Intel] Querying for {indicator_type}: {value}...")
    if indicator_type == "url" and "evil-domain.com" in value:
        return {"status": "malicious", "source": "Mock Intel Service"}
    if indicator_type == "ip" and value in KNOWN_BAD_IPS:
        return {"status": "malicious", "source": "Mock Intel Service"}
    if indicator_type == "hash" and value.startswith("8f"):
        return {"status": "malicious", "source": "Mock Intel Service", "malware_family": "GenericDownloader.exe"}
    return {"status": "unknown", "source": "Mock Intel Service"}

print("✅ Playbook setup and configuration loaded.")

In [ ]:
import json
import re
from datetime import datetime, timezone
import pandas as pd

# Configure pandas to display full URL text
pd.set_option('display.max_colwidth', None)

# --- Configuration (User to customize) ---
KNOWN_BAD_DOMAINS = {"evil-domain.com", "phish-corp.net"}
KNOWN_BAD_IPS = {"198.51.100.55"}
SUSPICIOUS_KEYWORDS = {"login", "verify", "account", "update", "urgent", "password"}

# --- Mock Threat Intelligence Function ---
def query_threat_intel_mock(indicator_type: str, value: str) -> dict:
    """Mocks a threat intelligence query. Replace with actual API calls."""
    print(f"[Intel] Querying for {indicator_type}: {value}...")
    if indicator_type == "url" and "evil-domain.com" in value:
        return {"status": "malicious", "source": "Mock Intel Service"}
    if indicator_type == "ip" and value in KNOWN_BAD_IPS:
        return {"status": "malicious", "source": "Mock Intel Service"}
    if indicator_type == "hash" and value.startswith("8f"):
        return {"status": "malicious", "source": "Mock Intel Service", "malware_family": "GenericDownloader.exe"}
    return {"status": "unknown", "source": "Mock Intel Service"}

print("✅ Playbook setup and configuration loaded.")

In [ ]:
def triage_phishing_alert(alert: dict) -> dict:
    """Performs triage, enrichment, and severity scoring on a phishing alert."""
    incident = {
        "incident_id": f"INC-{datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S')}",
        "type": "Phishing",
        "status": "New",
        "severity": "Low",
        "summary": alert.get("subject"),
        "triage_notes": []
    }    
    severity_score = 0
    indicators_list = []

    # Analyze sender, IP, URLs, Hashes
    sender_email = alert.get("sender", "")
    sender_domain = sender_email.split('@')[1] if '@' in sender_email else ""
    if sender_domain:
        intel = "On Blocklist" if sender_domain in KNOWN_BAD_DOMAINS else "Unknown"
        indicators_list.append({"type": "domain", "value": sender_domain, "intel": intel, "notes": ""})
        if intel == "On Blocklist": severity_score += 50
    
    sender_ip = alert.get("sender_ip", "")
    if sender_ip:
        intel_result = query_threat_intel_mock("ip", sender_ip)
        indicators_list.append({"type": "ip", "value": sender_ip, "intel": intel_result.get("status"), "notes": ""})
        if intel_result.get("status") == "malicious": severity_score += 25

    for url in alert.get("urls", []):
        intel_result = query_threat_intel_mock("url", url)
        note = "URL contains suspicious keywords." if any(keyword in url.lower() for keyword in SUSPICIOUS_KEYWORDS) else ""
        indicators_list.append({"type": "url", "value": url, "intel": intel_result.get("status"), "notes": note})
        if intel_result.get("status") == "malicious": severity_score += 50
        if note: severity_score += 10

    for att in alert.get("attachments", []):
        intel_result = query_threat_intel_mock("hash", att['sha256'])
        note = f"Filename: {att['filename']}"
        indicators_list.append({"type": "file_hash_sha256", "value": att['sha256'], "intel": intel_result.get("status"), "notes": note})
        if intel_result.get("status") == "malicious": severity_score += 75
    
    if any(keyword in alert.get("subject", "").lower() for keyword in SUSPICIOUS_KEYWORDS):
        severity_score += 10
        incident["triage_notes"].append("Subject line contains suspicious keywords.")

    # Finalize Severity
    if severity_score >= 100: incident["severity"] = "Critical"
    elif severity_score >= 75: incident["severity"] = "High"
    elif severity_score >= 40: incident["severity"] = "Medium"
        
    incident['indicators_df'] = pd.DataFrame(indicators_list).fillna('')
    incident['status'] = "Triaged"
    return incident

def generate_next_steps(incident: dict):
    """Generates recommended next steps based on incident details."""
    print("\n--- 🗣️ Analyst Briefing ---")
    print(f"Incident {incident['incident_id']} has been triaged with a severity of: {incident['severity']}")
    for note in incident.get('triage_notes', []):
        print(f"- {note}")
    
    print("\n--- 💡 Recommended Next Steps (Containment & Investigation) ---")
    steps = []
    df = incident['indicators_df']
    
    if incident["severity"] in ["High", "Critical"]:
        steps.append("[IMMEDIATE] Search mail logs for other recipients and delete emails.")
        if 'domain' in df.type.values: steps.append(f"[BLOCK] Sender domain(s): {list(df[df.type=='domain'].value)}")
        if 'ip' in df.type.values: steps.append(f"[BLOCK] Sender IP(s): {list(df[df.type=='ip'].value)}")
        if 'url' in df.type.values: steps.append(f"[BLOCK] URL(s): {list(df[df.type=='url'].value)}")
        if 'file_hash_sha256' in df.type.values: steps.append(f"[BLOCK/HUNT] File Hash(es): {list(df[df.type=='file_hash_sha256'].value)}")
        steps.append("[INVESTIGATE] Check proxy/firewall logs for any user access to the URL(s).")
        steps.append("[INVESTIGATE] Use EDR to hunt for file hash(es) on endpoints.")
    else:
        steps.append("Severity is below High. Continue to monitor. No immediate blocking action required.")

    for i, step in enumerate(steps, 1):
        print(f"{i}. {step}")

# --- Run the Triage and Generate Recommendations ---
triaged_incident = triage_phishing_alert(raw_alert)

print("--- 📊 Triage Summary ---")
display(triaged_incident['indicators_df'])
generate_next_steps(triaged_incident)